In [1]:
from fftarray import FFTDimension, shift_frequency, shift_position
from fftarray.plot.bokeh import plt_fft
import numpy as np
from bokeh.plotting import figure, show, row
from bokeh.io import output_notebook
output_notebook()

def gauss_pos(x, a, sigma):
    return (a * np.exp(-(x**2/(2.* sigma**2))))/(np.sqrt(2 * np.pi) * sigma)

def gauss_freq(f, a, sigma):
    return (a * np.exp(-(1/2)*(2*np.pi*f)**2*sigma**2))

Loading BokehJS ...

Initialize the dimension and get the basic x and f arrays.

In [3]:
dim = FFTDimension("x",
        pos_middle=1.,
        pos_extent = 10.,
        freq_middle = 2.5/(2*np.pi),
        freq_extent = 20./(2*np.pi),
        loose_params=["pos_extent", "freq_extent"]
    )

x = dim.pos_array()
f = dim.freq_array()

show(plt_fft(x))
show(plt_fft(f))

In [7]:
gauss_from_pos = gauss_pos(a = 1.2, x = x, sigma = 0.7)
gauss_from_freq = gauss_freq(a = 1.2, f = f, sigma = 0.7)

np.testing.assert_array_almost_equal(gauss_from_pos, gauss_from_freq.pos_array())
np.testing.assert_array_almost_equal(gauss_from_freq, gauss_from_pos.freq_array())

# TODO Plot titles?
show(plt_fft(gauss_from_pos))
show(plt_fft(gauss_from_freq))

Just compute a centered Gaussian in position and frequency space.
It does not matter where it is defined, the result is the same (up to numerical inaccuracies).

In [8]:
gauss_from_pos = shift_frequency(gauss_pos(a = 1.2, x = x, sigma = 0.7), {"x": 0.9})
gauss_from_freq = gauss_freq(a = 1.2, f = f - 0.9, sigma = 0.7)

np.testing.assert_array_almost_equal(gauss_from_pos, gauss_from_freq.pos_array())
np.testing.assert_array_almost_equal(gauss_from_freq, gauss_from_pos.freq_array())

show(plt_fft(gauss_from_pos))
show(plt_fft(gauss_from_freq))

Now test with a Gauss shifted in position space. The `shift_position` function multiplies a phase factor unto the `FFTArray` in frequency space to achieve a user.defined shift.

In [9]:
gauss_from_pos = gauss_pos(a = 1.2, x = x - 0.9, sigma = 0.7)
gauss_from_freq = shift_position(gauss_freq(a = 1.2, f = f, sigma = 0.7), {"x": 0.9})

np.testing.assert_array_almost_equal(gauss_from_pos, gauss_from_freq.pos_array())
np.testing.assert_array_almost_equal(gauss_from_freq, gauss_from_pos.freq_array())

show(plt_fft(gauss_from_pos))
show(plt_fft(gauss_from_freq))